## Part 4: Generation 

In [1]:
import bs4,os
from langchain_community.document_loaders import WebBaseLoader
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ))

blog_docs = loader.load()

In [2]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("langchain_api_key")
os.environ["LANGCHAIN_PROJECT"] = "RAG"

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

text = text_splitter.split_documents(blog_docs)

In [4]:
embedding_function = CohereEmbeddings(model="embed-english-light-v3.0",cohere_api_key=os.getenv("cohere_api_key2"))

In [5]:
db = Chroma.from_documents(persist_directory="./chroma_db72",embedding=embedding_function,documents=text)

In [10]:
vectorstore = Chroma(persist_directory="./chroma_db72", embedding_function=embedding_function)
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",google_api_key=os.getenv("google_api_key"))
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser())

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller, simpler steps. This can be done using various methods, including:\n\n* **Chain of Thought (CoT):** This technique prompts the model to "think step by step" to decompose complex tasks into smaller steps. \n* **Tree of Thoughts:** This extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure for potential solutions.\n* **LLM with simple prompting:**  The LLM can be prompted with questions like "Steps for XYZ." or "What are the subgoals for achieving XYZ?".\n* **Task-specific instructions:**  Instructions tailored to the specific task, such as "Write a story outline" for writing a novel.\n* **Human inputs:**  Direct input from a human can also be used to decompose tasks. \n'

In [15]:
question = "What is Task Decomposition?"
context = retriever.get_relevant_documents(question)

template = f"""Answer the question based only on the following context:
{context}

Question: {question}
"""
print(template)

Answer the question based only on the following context:
[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-firs

In [17]:
print(llm.invoke(template).content)

Task Decomposition is the process of breaking down a complex task into smaller, simpler steps. 

